In [1]:
from analyzer import Analyzer
from tools import *
import matplotlib.pyplot as plt
from scipy.io import savemat
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [18]:
vi_data = {
    'dataset': [],
    'num': [],
    'vi': [],
}

for dataset in ['dogs3', 'birds5']:
    num_vs_time_per_question = {}
    demo_time = []
    nums = []
    num_vs_error = []
    for num in [2, 3, 4, 5, 6, 7, 8]:
        file = f'sd-rq-wp-{num}-{dataset}-2023'
        np.random.seed(0)
        analyer = Analyzer("./data/{}.json".format(file),
                           size_of_dataset(dataset), retrieve_true_adj_matrix(dataset)(), radio=False, dataset=dataset, practice=True)
        analyer.raw_data['demo'].demo_completion_time
        time_per_question = analyer.times
        demo_completion_times = analyer.demo_times

        workers_edge_error_rate_first_half = []
        workers_edge_error_rate_second_half = []

        for answers in analyer.workers_answers:
            true_adj_mat = answers.true_adj_matrix

            num_correct_first_half = 0
            num_incorrect_first_half = 0
            num_correct_second_half = 0
            num_incorrect_second_half = 0
            halves = len(answers.worker_answers) // 2

            for answer in answers.worker_answers[:halves]:
                for cluster in answer.clusters:
                    for i, j in combinations(sorted(cluster), 2):
                        if true_adj_mat[i, j] == 1:
                            num_correct_first_half += 1
                        else:
                            num_incorrect_first_half += 1
                for cluster_x, cluster_y in combinations(answer.clusters, 2):
                    for i, j in product(cluster_x, cluster_y):
                        if true_adj_mat[i, j] == 0:
                            num_correct_first_half += 1
                        else:
                            num_incorrect_first_half += 1

            for answer in answers.worker_answers[halves:]:
                for cluster in answer.clusters:
                    for i, j in combinations(sorted(cluster), 2):
                        if true_adj_mat[i, j] == 1:
                            num_correct_second_half += 1
                        else:
                            num_incorrect_second_half += 1
                for cluster_x, cluster_y in combinations(answer.clusters, 2):
                    for i, j in product(cluster_x, cluster_y):
                        if true_adj_mat[i, j] == 0:
                            num_correct_second_half += 1
                        else:
                            num_incorrect_second_half += 1
            
            edge_error_rate_fist_half = num_incorrect_first_half / (num_correct_first_half + num_incorrect_first_half)
            edge_error_rate_second_half = num_incorrect_second_half / (num_correct_second_half + num_incorrect_second_half)
            workers_edge_error_rate_first_half.append(edge_error_rate_fist_half)
            workers_edge_error_rate_second_half.append(edge_error_rate_second_half)

        print(np.mean(workers_edge_error_rate_first_half))
        print(np.mean(workers_edge_error_rate_second_half))
        
        break
        number_correct_edges = list(map(lambda x: x.worker_edge_correct, analyer.workers_answers))
        total_number_edges = list(map(lambda x: x.worker_edge_correct + x.worker_edge_incorrect, analyer.workers_answers))
        num_0 = list(map(lambda x: x.num_0, analyer.workers_answers))
        num_0_correct = list(map(lambda x: x.num_0 - x.num_0_to_1, analyer.workers_answers))
        num_1 = list(map(lambda x: x.num_1, analyer.workers_answers))
        num_1_correct = list(map(lambda x: x.num_1 - x.num_1_to_0, analyer.workers_answers))

        for i in range(len(number_correct_edges)):
            _data = [num]
            _data.append(number_correct_edges[i])
            _data.append(total_number_edges[i])
            _data.append(num_1[i])
            _data.append(num_1_correct[i])
            _data.append(num_0[i])
            _data.append(num_0_correct[i])
            num_vs_error.append(_data)

        worker_edge_mat = np.array([
            number_correct_edges,
            total_number_edges,
            num_1,
            num_1_correct,
            num_0,
            num_0_correct
        ]).T

        num_vs_time_per_question[num] = time_per_question
        for demo_completion_time in demo_completion_times:
            demo_time.append(demo_completion_time)
            nums.append(num)
        savemat("./mats/{}_worker_edge_mat.mat".format(file), {"worker_edge_mat": worker_edge_mat})
        
        
        true_labels = retrieve_true_labels(dataset)()
        predicted_labels = analyer.spectral_clustering(6 if dataset == 'birds5' else 3)
        VI = vi(predicted_labels=predicted_labels, true_labels=true_labels)
        
        vi_data['vi'].append(VI)
        vi_data['num'].append(num)
        vi_data['dataset'].append(dataset)
    
    break
    num_vs_error_df = pd.DataFrame(num_vs_error, columns=["num_images_per_question", "num_correct_edges", "num_edges", "num_1", "num_1_correct", "num_0", "num_0_correct"])
    num_vs_error_df.to_csv(f'./csv/edge_error_per_worker_{dataset}_drag_and_drop.csv', index=False)
    
    x = sum(map(lambda x: [x[0] for _ in range(len(x[1]))], num_vs_time_per_question.items()), [])
    y = sum(num_vs_time_per_question.values(), [])
    
    data = pd.DataFrame({"num_images_per_question": x, "time_per_question": y})
    data.to_csv(f'./csv/time_per_question_{dataset}_drag_and_drop.csv', index=False)
    
    # pd.DataFrame({'num': nums, 'practice_error': analyer.practice_error}).to_csv(
    #     f'./csv/practice_error_{dataset}_drag_and_drop.csv', index=False)
    # pd.DataFrame({'num': nums, 'practice_attempts': analyer.practice_attempts}).to_csv(
    #     f'./csv/practice_attempts_{dataset}_drag_and_drop.csv', index=False)
    # pd.DataFrame({'num': nums, 'practice_completion_time': analyer.practice_total_times}).to_csv(
    #     f'./csv/practice_completion_time_{dataset}_drag_and_drop.csv', index=False)
    # pd.DataFrame({'num': nums, 'demo_completion_time': demo_time}).to_csv(
    #     f'./csv/demo_completion_time_{dataset}_drag_and_drop.csv', index=False)
pd.DataFrame(vi_data).to_csv('./csv/vi_drag_and_drop.csv', index=False)

0.2668888888888889
0.25022222222222223


In [ ]:
vi_data = {
    'dataset': [],
    'num': [],
    'vi': [],
}

for dataset in ['dogs3', 'birds5']:
    
    num_vs_time_per_question = {}
    num_vs_error = []
    
    for num in [2, 3, 4]:
        file = f'radio-{num}-image-{dataset}-2023'
        np.random.seed(0)
        analyer = Analyzer("./data/{}.json".format(file),
                           size_of_dataset(dataset), retrieve_true_adj_matrix(dataset)(), radio=True, dataset=dataset, practice=False)
        time_per_question = analyer.times
        number_correct_edges = list(map(lambda x: x.worker_edge_correct, analyer.workers_answers))
        total_number_edges = list(map(lambda x: x.worker_edge_correct + x.worker_edge_incorrect, analyer.workers_answers))
        num_0 = list(map(lambda x: x.num_0, analyer.workers_answers))
        num_0_correct = list(map(lambda x: x.num_0 - x.num_0_to_1, analyer.workers_answers))
        num_1 = list(map(lambda x: x.num_1, analyer.workers_answers))
        num_1_correct = list(map(lambda x: x.num_1 - x.num_1_to_0, analyer.workers_answers))

        for i in range(len(number_correct_edges)):
            _data = [num]
            _data.append(number_correct_edges[i])
            _data.append(total_number_edges[i])
            _data.append(num_1[i])
            _data.append(num_1_correct[i])
            _data.append(num_0[i])
            _data.append(num_0_correct[i])
            num_vs_error.append(_data)

        worker_edge_mat = np.array([
            number_correct_edges,
            total_number_edges,
            num_1,
            num_1_correct,
            num_0,
            num_0_correct
        ]).T

        num_vs_time_per_question[num] = time_per_question
        savemat("./mats/{}_worker_edge_mat.mat".format(file), {"worker_edge_mat": worker_edge_mat})
        
        true_labels = retrieve_true_labels(dataset)()
        predicted_labels = analyer.spectral_clustering()
        if dataset == "birds5":
            true_labels = true_labels[:-50]
        VI = vi(predicted_labels=predicted_labels, true_labels=true_labels)
        
        vi_data['vi'].append(VI)
        vi_data['num'].append(num)
        vi_data['dataset'].append(dataset)
    
    num_vs_error_df = pd.DataFrame(num_vs_error, columns=["num_images_per_question", "num_correct_edges", "num_edges", "num_1", "num_1_correct", "num_0", "num_0_correct"])
    num_vs_error_df.to_csv(f'./csv/edge_error_per_worker_{dataset}_radio.csv', index=False)
    
    x = sum(map(lambda x: [x[0] for _ in range(len(x[1]))], num_vs_time_per_question.items()), [])
    y = sum(num_vs_time_per_question.values(), [])
    
    data = pd.DataFrame({"num_images_per_question": x, "time_per_question": y})
    data.to_csv(f'./csv/time_per_question_{dataset}_radio.csv', index=False)
pd.DataFrame(vi_data).to_csv('./csv/vi_radio.csv')